# Arctic

> Arctic helper scripts and functions.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# | default_exp arctic

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export
import os

import click
from arcticdb import Arctic, LibraryOptions
import hydra
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
from pathlib import Path
from lobster_tools.config import MainConfig, Overrides, NASDAQExchange, register_configs, get_config
from lobster_tools.preprocessing import Data, Lobster
import sys
import pandas as pd
from logging import Logger
from datetime import date

ImportError: cannot import name 'NASDAQExchange' from 'lobster_tools.config' (/nfs/home/nicolasp/home/code/lobster-tools/notebooks/lobster_tools/config.py)

When using the `@hydra.main` decorator, `register_configs` must be called. If simply using a notebook or writing the CLIs with `click`, it is enough to use `get_config`.

In [ ]:
# from lobster_tools.config import XLEUniverseConfig, SimpleHyperparametersConfig

In [ ]:
# XLEUniverseConfig().equities
# SimpleHyperparametersConfig()

In [ ]:
# | export
# register_configs()
cfg = get_config(overrides=Overrides.full_server)
cfg = get_config(overrides=['data_config=server', 'hyperparameters=full', 'universe=SPY'])

In [ ]:
# | export

CONTEXT_SETTINGS = dict(
    help_option_names=["-h", "--help"],
    token_normalize_func=lambda x: x.lower() if isinstance(x, str) else x,
    show_default=True,
)

In [ ]:
# | export

def get_arctic_library(db_path, library):
    conn = f"lmdb://{db_path}"
    arctic = Arctic(conn)
    arctic_library = arctic[library]
    return arctic_library

In [ ]:
@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
def arctic_list_symbols(db_path, library) -> None:
    """List symbols in the arcticdb library."""
    arctic_library = get_arctic_library(db_path=db_path, library=library)
    print(f"Symbols in library {library}")
    print(arctic_library.list_symbols())

In [ ]:
# | exports
# | code-fold: true

@click.command(context_settings=CONTEXT_SETTINGS)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
@click.option("-t", "--ticker", required=True, help="ticker to print")
@click.option("-s", "--start_date", default="2020-01-02", help="start date")
@click.option("-e", "--end_date", default="2020-01-07", help="end date")
def arctic_read_symbol(db_path, library, ticker, start_date, end_date,
):
    """Print df.head() and available columns for ticker in arcticdb library."""
    arctic_library = get_arctic_library(db_path=db_path, library=library)

    start_datetime = pd.Timestamp(f"{start_date}T{NASDAQExchange.exchange_open}")
    end_datetime = pd.Timestamp(f"{end_date}T{NASDAQExchange.exchange_close}")
    date_range = (start_datetime, end_datetime)
    df = arctic_library.read(ticker, date_range=date_range).data
    
    print(f"Printing df.head() for ticker {ticker}")
    print(df.tail())

Write to arctic again

In [ ]:
# | exports
# | code-fold: true


@click.command(context_settings=CONTEXT_SETTINGS)
@click.option(
    "-c", "--csv_path", default=cfg.data_config.csv_files_path, help="csv files path"
)
@click.option("-d", "--db_path", default=cfg.db.db_path, help="database path")
@click.option("-l", "--library", default=cfg.db.library, help="library name")
@click.option("-t", "--ticker", required=True, help="ticker to write to db")
@click.option("-s", "--start_date", default="2020-01-01", help="start date")
@click.option("-e", "--end_date", default="2020-02-01", help="end date")
def arctic_write_symbol(
    db_path,
    library,
    csv_path,
    ticker,
    start_date,
    end_date,
):
    arctic_library = get_arctic_library(db_path=db_path, library=library)

    if ticker in arctic_library.list_symbols():
        print("warning - there is already data for ths ticker")
        user_input = input("Proceed by adding data to this symbol? (y/n): ")
        user_input = user_input.lower()
        match user_input:
            case "y":
                pass
            case "n":
                sys.exit(0)
            case _:
                sys.exit(1)

    date_range = (start_date, end_date)
    data = Data(
        directory_path=csv_path,
        ticker=ticker,
        date_range=date_range,
        aggregate_duplicates=False,
    )
    lobster = Lobster(data=data)
    df = pd.concat([lobster.messages, lobster.book], axis=1)
    print(df)

    arctic_library.append(symbol=ticker, data=df)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()